In [1]:
# NOTE: this is a custom cell that contains the common imports I personally 
# use these may/may not be necessary for the following examples

# DL framework
import tensorflow as tf

from datetime import datetime

# common packages
import numpy as np
import os # handling file i/o
import sys
import math
import time # timing epochs

# for ordered dict when building layer components
import collections

# plotting pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib import colors # making colors consistent
from mpl_toolkits.axes_grid1 import make_axes_locatable # colorbar helper

# read image
from imageio import imread
# + data augmentation
from scipy import ndimage
from scipy import misc

# used for manually saving best params
import pickle

# for shuffling data batches
from sklearn.utils import shuffle

# const
SEED = 42

# Helper to make the output consistent
def reset_graph(seed=SEED):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# helper to create dirs if they don't already exist
def maybe_create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print("{} created".format(dir_path))
    else:
        print("{} already exists".format(dir_path))
    
# set tf log level to supress messages, unless an error
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Important Version information
print("Python: {}".format(sys.version_info[:]))
print('TensorFlow: {}'.format(tf.__version__))

# Check if using GPU
if not tf.test.gpu_device_name():
    print('No GPU')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    
reset_graph()

Python: (3, 5, 4, 'final', 0)
TensorFlow: 1.6.0-dev20180105
No GPU


In [2]:
features = np.arange(20)
labels = np.random.randint(2, size=len(features))
print(features)
print(labels)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 0]


In [3]:
# convert to tensor
X = tf.convert_to_tensor(features, np.float32)
y = tf.convert_to_tensor(labels, np.int8)

In [4]:
# params

# number of epochs
# > used to set the number of times to loop the dataset
n_epochs = 4
# size of each batch
batch_size = 3
# size of the "shuffle buffer", meaning if the buffer size is 
# 2, the data in place a in [a,b,c,d,e,f,g] the data can only be
# shuffled into 1 of 2 locations (current | one away){a or b}.
# if the buffer_size was 1, then the data could only be shuffled 5 places away
buffer_size = 5

In [5]:
def return_batched_iter(X, y):
    reset_graph()

    # create dataset
    dataset = tf.data.Dataset.from_tensor_slices((X, y))

    # set batch size
    dataset = dataset.batch(batch_size)

    # set number of times to loop data
    dataset = dataset.repeat(n_epochs)

    iterator = dataset.make_one_shot_iterator()
    next_batch = iterator.get_next()
    
    return next_batch

In [6]:
# standard
next_batch = return_batched_iter(X, y)

with tf.Session() as sess:
    for e in range(n_epochs):
        print("e: {}".format(e))
        for i in range((len(features)//batch_size) + 1):
            print("i: {} => {}".format(i, sess.run(next_batch)))

e: 0
i: 0 => (array([ 0.,  1.,  2.], dtype=float32), array([0, 1, 0], dtype=int8))
i: 1 => (array([ 3.,  4.,  5.], dtype=float32), array([0, 0, 1], dtype=int8))
i: 2 => (array([ 6.,  7.,  8.], dtype=float32), array([0, 0, 0], dtype=int8))
i: 3 => (array([  9.,  10.,  11.], dtype=float32), array([1, 0, 0], dtype=int8))
i: 4 => (array([ 12.,  13.,  14.], dtype=float32), array([0, 0, 1], dtype=int8))
i: 5 => (array([ 15.,  16.,  17.], dtype=float32), array([0, 1, 1], dtype=int8))
i: 6 => (array([ 18.,  19.], dtype=float32), array([1, 0], dtype=int8))
e: 1
i: 0 => (array([ 0.,  1.,  2.], dtype=float32), array([0, 1, 0], dtype=int8))
i: 1 => (array([ 3.,  4.,  5.], dtype=float32), array([0, 0, 1], dtype=int8))
i: 2 => (array([ 6.,  7.,  8.], dtype=float32), array([0, 0, 0], dtype=int8))
i: 3 => (array([  9.,  10.,  11.], dtype=float32), array([1, 0, 0], dtype=int8))
i: 4 => (array([ 12.,  13.,  14.], dtype=float32), array([0, 0, 1], dtype=int8))
i: 5 => (array([ 15.,  16.,  17.], dtype=floa

## Shuffle all elements

In [7]:
def return_batched_iter_full_shuffle(X, y):
    reset_graph()

    # create dataset
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    
    # shuffle entire set
    dataset = dataset.shuffle(buffer_size=buffer_size)

    # set batch size
    dataset = dataset.batch(batch_size)

    # set number of times to loop data
    dataset = dataset.repeat(n_epochs)

    iterator = dataset.make_one_shot_iterator()
    next_batch = iterator.get_next()
    
    return next_batch

In [8]:
next_batch = return_batched_iter_full_shuffle(X, y)

l = [x for x in zip(features,labels)]
for li in l:
    print(li)

with tf.Session() as sess:
    for e in range(n_epochs):
        print("e: {}".format(e))
        for i in range((len(features)//batch_size) + 1):
            print("i: {} => {}".format(i, sess.run(next_batch)))

(0, 0)
(1, 1)
(2, 0)
(3, 0)
(4, 0)
(5, 1)
(6, 0)
(7, 0)
(8, 0)
(9, 1)
(10, 0)
(11, 0)
(12, 0)
(13, 0)
(14, 1)
(15, 0)
(16, 1)
(17, 1)
(18, 1)
(19, 0)
e: 0
i: 0 => (array([ 1.,  5.,  0.], dtype=float32), array([1, 1, 0], dtype=int8))
i: 1 => (array([ 3.,  8.,  6.], dtype=float32), array([0, 0, 0], dtype=int8))
i: 2 => (array([ 10.,   7.,  12.], dtype=float32), array([0, 0, 0], dtype=int8))
i: 3 => (array([  4.,  11.,  13.], dtype=float32), array([0, 0, 0], dtype=int8))
i: 4 => (array([  2.,  16.,  17.], dtype=float32), array([0, 1, 1], dtype=int8))
i: 5 => (array([ 18.,  19.,   9.], dtype=float32), array([1, 0, 1], dtype=int8))
i: 6 => (array([ 15.,  14.], dtype=float32), array([0, 1], dtype=int8))
e: 1
i: 0 => (array([ 3.,  5.,  1.], dtype=float32), array([0, 1, 1], dtype=int8))
i: 1 => (array([ 4.,  0.,  6.], dtype=float32), array([0, 0, 0], dtype=int8))
i: 2 => (array([  8.,  10.,   9.], dtype=float32), array([0, 0, 1], dtype=int8))
i: 3 => (array([  2.,  14.,  12.], dtype=float32), 

## Shuffle between batches
NOTICE: The only difference between this example and the example above is the order in which the methods are called -- here, the `shuffle` is called after the `batch` size is set and so batches are shuffled around, not the individual batches.

In [9]:
def return_batched_iter_betweenbatches_shuffle(X, y):
    reset_graph()

    # create dataset
    dataset = tf.data.Dataset.from_tensor_slices((X, y))

    # set batch size
    dataset = dataset.batch(batch_size)

    # set number of times to loop data
    dataset = dataset.repeat(n_epochs)
    
    # shuffle between all batches (among all epochs)
    dataset = dataset.shuffle(buffer_size=buffer_size)

    iterator = dataset.make_one_shot_iterator()
    next_batch = iterator.get_next()
    
    return next_batch

In [10]:
next_batch = return_batched_iter_betweenbatches_shuffle(X, y)
with tf.Session() as sess:
    for e in range(n_epochs):
        print("e: {}".format(e))
        for i in range((len(features)//batch_size) + 1):
            print("i: {} => {}".format(i, sess.run(next_batch)))

e: 0
i: 0 => (array([ 3.,  4.,  5.], dtype=float32), array([0, 0, 1], dtype=int8))
i: 1 => (array([ 12.,  13.,  14.], dtype=float32), array([0, 0, 1], dtype=int8))
i: 2 => (array([  9.,  10.,  11.], dtype=float32), array([1, 0, 0], dtype=int8))
i: 3 => (array([ 6.,  7.,  8.], dtype=float32), array([0, 0, 0], dtype=int8))
i: 4 => (array([ 15.,  16.,  17.], dtype=float32), array([0, 1, 1], dtype=int8))
i: 5 => (array([ 18.,  19.], dtype=float32), array([1, 0], dtype=int8))
i: 6 => (array([ 3.,  4.,  5.], dtype=float32), array([0, 0, 1], dtype=int8))
e: 1
i: 0 => (array([ 0.,  1.,  2.], dtype=float32), array([0, 1, 0], dtype=int8))
i: 1 => (array([  9.,  10.,  11.], dtype=float32), array([1, 0, 0], dtype=int8))
i: 2 => (array([ 0.,  1.,  2.], dtype=float32), array([0, 1, 0], dtype=int8))
i: 3 => (array([ 12.,  13.,  14.], dtype=float32), array([0, 0, 1], dtype=int8))
i: 4 => (array([ 15.,  16.,  17.], dtype=float32), array([0, 1, 1], dtype=int8))
i: 5 => (array([ 6.,  7.,  8.], dtype=floa

## Shuffle batches within epoch
This shouldn't be surprising based on the logic outlined above

In [11]:
def return_batched_iter_betweenEpochBatches_shuffle(X, y):
    reset_graph()

    # create dataset
    dataset = tf.data.Dataset.from_tensor_slices((X, y))

    # set batch size
    dataset = dataset.batch(batch_size)
    
    # shuffle between batches
    dataset = dataset.shuffle(buffer_size=buffer_size)

    # set number of times to loop data
    dataset = dataset.repeat(n_epochs)

    iterator = dataset.make_one_shot_iterator()
    next_batch = iterator.get_next()
    
    return next_batch

In [12]:
next_batch = return_batched_iter_betweenEpochBatches_shuffle(X, y)
with tf.Session() as sess:
    for e in range(n_epochs):
        print("e: {}".format(e))
        for i in range((len(features)//batch_size) + 1):
            print("i: {} => {}".format(i, sess.run(next_batch)))

e: 0
i: 0 => (array([ 0.,  1.,  2.], dtype=float32), array([0, 1, 0], dtype=int8))
i: 1 => (array([ 6.,  7.,  8.], dtype=float32), array([0, 0, 0], dtype=int8))
i: 2 => (array([ 12.,  13.,  14.], dtype=float32), array([0, 0, 1], dtype=int8))
i: 3 => (array([ 3.,  4.,  5.], dtype=float32), array([0, 0, 1], dtype=int8))
i: 4 => (array([  9.,  10.,  11.], dtype=float32), array([1, 0, 0], dtype=int8))
i: 5 => (array([ 18.,  19.], dtype=float32), array([1, 0], dtype=int8))
i: 6 => (array([ 15.,  16.,  17.], dtype=float32), array([0, 1, 1], dtype=int8))
e: 1
i: 0 => (array([ 6.,  7.,  8.], dtype=float32), array([0, 0, 0], dtype=int8))
i: 1 => (array([ 12.,  13.,  14.], dtype=float32), array([0, 0, 1], dtype=int8))
i: 2 => (array([ 0.,  1.,  2.], dtype=float32), array([0, 1, 0], dtype=int8))
i: 3 => (array([ 15.,  16.,  17.], dtype=float32), array([0, 1, 1], dtype=int8))
i: 4 => (array([  9.,  10.,  11.], dtype=float32), array([1, 0, 0], dtype=int8))
i: 5 => (array([ 18.,  19.], dtype=float32